In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

# **데이터셋 로드 & 알아볼 항목 분할**

In [3]:
df2 =  pd.read_csv('/content/drive/MyDrive/last2.csv')


,product_id,category_id,brand,price,user_id,division1,division2,rating
0,1307067,2053013558920217191,lenovo,251.74,550050854,computers,notebook,1.5
1,1004237,2053013555631882655,apple,1081.98,535871217,electronics,smartphone,1.5
2,3900746,2053013552326770905,haier,102.38,555444559,appliances,environment,1.5
3,1306631,2053013558920217191,hp,580.89,550050854,computers,notebook,1.5
4,1005135,2053013555631882655,apple,1747.79,535871217,electronics,smartphone,1.5
...,...,...,...,...,...,...,...,...
14333031,2702331,2053013563911439225,lg,527.43,524356542,appliances,kitchen,1.5
14333032,2300275,2053013560530830019,gopro,527.40,537931532,electronics,camera,1.5
14333033,10800172,2053013554994348409,redmond,61.75,527322328,unknown,unknown,1.5
14333034,21407424,2053013561579406073,tissot,689.85,513118352,electronics,clocks,1.5


In [6]:
df_cat = df2.loc[(df2['division2']=='smartphone')|
                    (df2['division2']=='video')|
                    (df2['division2']=='kitchen')|
                    (df2['division2']=='notebook')
                    ]

df_cat=df_cat.reset_index(drop=True)
df_cat

,product_id,category_id,brand,price,user_id,division1,division2,rating
0,1307067,2053013558920217191,lenovo,251.74,550050854,computers,notebook,1.5
1,1004237,2053013555631882655,apple,1081.98,535871217,electronics,smartphone,1.5
2,1306631,2053013558920217191,hp,580.89,550050854,computers,notebook,1.5
3,1005135,2053013555631882655,apple,1747.79,535871217,electronics,smartphone,1.5
4,2700239,2053013563911439225,atlant,283.12,521242564,appliances,kitchen,1.5
...,...,...,...,...,...,...,...,...
6721646,1004565,2053013555631882655,huawei,164.95,566143627,electronics,smartphone,1.5
6721647,16700260,2053013559901684381,unknown,31.64,566255262,furniture,kitchen,1.5
6721648,1002786,2053013555631882655,unknown,391.26,512789086,electronics,smartphone,1.5
6721649,1005008,2053013555631882655,xiaomi,94.46,557202408,electronics,smartphone,1.5


# **Surprise 모델 사용**

In [10]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import pandas as pd
from surprise import SVD, BaselineOnly, SVDpp, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline
from surprise.model_selection import GridSearchCV
from surprise import KNNBasic
from surprise import accuracy
from surprise import dump
from surprise.model_selection import KFold
from surprise import NormalPredictor
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise.model_selection import train_test_split


데이터셋 나누기

In [12]:
# surprise 모듈의 훈련,테스트 모듈 나누는것과 비슷한 방법으로 데이터 분리 진행

# train, test split

# 데이터 랜덤하게 섞기
df_r_m = df_cat.sample(frac=1)

# 7:3 비율로 나누기
cutoff = int(.7 * len(df_r_m))
train_r = df_r_m[:cutoff]
test_r = df_r_m[cutoff:]


train_r = train_r[['user_id', 'product_id', 'rating', 'division2', 'brand']]
test_r = test_r[['user_id', 'product_id', 'rating', 'division2', 'brand']]
print("train_size:", len(train_r), "test_size:", len(test_r))

train_r.head()

train_size: 4705155 test_size: 2016496


,user_id,product_id,rating,division2,brand
1277629,543500703,1004903,3.1,smartphone,huawei
814203,546862281,1004650,1.5,smartphone,samsung
5926385,518316420,1307070,1.5,notebook,acer
6452097,550829569,16900018,1.5,kitchen,unknown
3453235,550788138,1002633,1.5,smartphone,apple


In [13]:
def convert_traintest_dataframe_forsurprise(training_dataframe, testing_dataframe):
    reader = Reader(rating_scale=(1, 5))
    trainset = Dataset.load_from_df(training_dataframe[['user_id', 'product_id', 'rating']], reader)
    testset = Dataset.load_from_df(testing_dataframe[['user_id', 'product_id', 'rating']], reader)
    
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    testset = testset.construct_testset(testset.raw_ratings)
    
    return trainset, testset

trainset, testset = convert_traintest_dataframe_forsurprise(train_r, test_r)

surprise 사용법: 


**surprise 라이브러리

test()는 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드. 즉 입력된 데이터 세트에 대해 추천 데이터 세트를 만들어준다.

predict()는 개별 사용자와 영화에 대한 추천 평점을 반환해준다.

uid(개별 사용자 아이디), 아이템(iid), 실제 평점(r_ui), 정보에 기반한 surprise의 추천 예측 평점(est)

details 속성은 내부 처리 시 추천 예측을 할 수 없는 경우에 로그용으로 데이터를 남기는데 사용된다. (was_impossible이 True이면 예측값을 생성할 수 없는 데이터 라는 의미)

리스트 객체 내에 내포된 Prediction 개체의 uid, iid, r_ri, est 등의 속성에 접근하려면 객체명.uid와 같은 형식으로 가능하다.



**predict()를 사용한 예측 

개별 사용자의 아이템에 대한 추천 평점을 예측해준다. 

인자로 개별 사용자 아이디, 아이템 아이디를 입력하면 추천 예측 평점을 포함한 정보를 반환한다. (사용자 아이디, 아이템 아이디는 문자열로 입력해야 한다)

# **KNN 모델과 SVD 모델**

In [14]:
# basic collaborative filtering algorithm taking into account a baseline rating.
# content-based filtering (CBF)

sim_options = {'name': 'cosine',   # 코사인 유사도 기반
               'user_based': False  # 아이템 기반 similarity
               }
knnbaseline_algo_i = KNNBaseline(sim_options=sim_options)

knnbaseline_algo_i.fit(trainset)
knnbaseline_predictions_i = knnbaseline_algo_i.test(testset)

file_name = 'KnnBaseline_model_i'
dump.dump(file_name, algo=knnbaseline_predictions_i)
# _, loaded_algo = dump.load(file_name)


# RSME 와 MAE 수치
accuracy.rmse(knnbaseline_predictions_i)
accuracy.mae(knnbaseline_predictions_i)
print("Done!")

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.1751
MAE:  0.0686
Done!


In [26]:
# 테스트 데이터 세트 전체에 대해 추천 아이템 평점 데이터를 생성 후 5개만 추출함
print('prediction type :', type(knnbaseline_predictions_i), ' size:', len(knnbaseline_predictions_i))
print('prediction 결과의 최초 5개 추출')
knnbaseline_predictions_i[:5]

prediction type : <class 'list'>  size: 2016496
prediction 결과의 최초 5개 추출


[Prediction(uid=523908254, iid=12600028, r_ui=1.5, est=1.456587773163651, details={'actual_k': 21, 'was_impossible': False}),
 Prediction(uid=513933577, iid=1002542, r_ui=1.5, est=1.5106195335115054, details={'actual_k': 35, 'was_impossible': False}),
 Prediction(uid=551290536, iid=1004726, r_ui=1.5, est=1.4584726521316977, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=519194946, iid=1004750, r_ui=1.5, est=1.5541702663454455, details={'actual_k': 16, 'was_impossible': False}),
 Prediction(uid=557394658, iid=1003549, r_ui=3.1, est=3.058400428154637, details={'actual_k': 40, 'was_impossible': False})]

In [16]:
# uid, iid, r_ui(실제 평점) ,est(예측평점) 추출

[(pred.uid, pred.iid, pred.est) for pred in knnbaseline_predictions_i[:3]]

[(523908254, 12600028, 1.456587773163651),
 (513933577, 1002542, 1.5106195335115054),
 (551290536, 1004726, 1.4584726521316977)]

In [31]:
# uid = 564085761 에 대한 유사도 높은 예측 iid, est 조회. 

pred_df = pd.DataFrame(knnbaseline_predictions_i)
pred_df[(pred_df['uid']==564085761)]

,uid,iid,r_ui,est,details
438991,564085761,1004258,1.5,1.527637,"{'actual_k': 12, 'was_impossible': False}"
1536591,564085761,1004259,1.5,1.553151,"{'actual_k': 12, 'was_impossible': False}"
1996151,564085761,1004258,1.5,1.527637,"{'actual_k': 12, 'was_impossible': False}"


In [32]:
# 임의의 user와 item을 입력했을 때 추천평점

user_id = str(518689178)
item_id = str(1004709)

pred = knnbaseline_algo_i.predict(user_id, item_id)
print(pred)

user: 518689178  item: 1004709    r_ui = None   est = 1.83   {'was_impossible': False}


In [33]:
# '유저가 확인한 item 수 / 추천 item 수 / 전체 item 수' 반환하는 함수

def get_unseen_surprise(df, userID):
    # 입력값으로 들어온 userID에 해당하는 사용자가 봤던 모든 item을 리스트로 생성
    seen_items = df[df['user_id']==userID]['product_id'].tolist()

    # 모든 아이템의 product_id를 리스트로 생성.
    total_items = df['product_id'].tolist()

    # 모든 아이템의 product_id 중 이미 봤던 아이템의 ID를 제외한 후 리스트로 생성
    unseen_items = [item for item in total_items if item not in seen_items]
    print('봤던 아이템 수:', len(seen_items), '추천 대상 아이템 수:', len(unseen_items), 
          '전체 아이템 수:', len(total_items))
    
    return unseen_items

# 유저 '550346297'의 결과

unseen_items = get_unseen_surprise(df_cat, 550346297)

봤던 아이템 수: 15 추천 대상 아이템 수: 6631739 전체 아이템 수: 6721651


In [44]:
# surprise 모델 활용한 10개의 추천 아이템
# df를 직접 넣어주는 경우

def recomm_item_by_surprise(algo, df, userID, unseen_items, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 본 적이 없는 아이템에 반복 수행한 후 결과를 list로 저장
    predictions = [algo.predict(str(userID), str(item_id)) for item_id in unseen_items]

    def sortkey_est(pred):
        return pred.est

    # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]

    # top_n으로 추출된 아이템의 정보 추출. user_id, 추천 예상 평점, category_id 추출
    top_item_ids = [int(pred.iid) for pred in top_predictions]
    top_item_rating = [ pred.est for pred in top_predictions]
    top_item_brands = df[df.product_id.isin(top_item_ids)]['brand']

    top_item_preds = [ (id, brand, rating) for id, brand, rating in \
                        zip(top_item_ids, top_item_brands, top_item_rating)]

    return top_item_preds

# 유저 '550346297'의 결과
unseen_items = get_unseen_surprise(df_cat, 550346297)
top_item_preds = recomm_item_by_surprise(knnbaseline_algo_i, df_cat, 550346297, unseen_items, top_n=10)


봤던 아이템 수: 15 추천 대상 아이템 수: 6631739 전체 아이템 수: 6721651


In [45]:
print('#### 10개의 추천 아이템 리스트 ####')
for top_item in top_item_preds:
    print("product_id :", top_item[0], '/', "brand:",top_item[1] )

#### 10개의 추천 아이템 리스트 ####
product_id : 1307067 / brand: lenovo
product_id : 1004237 / brand: apple
product_id : 1306631 / brand: hp
product_id : 1005135 / brand: apple
product_id : 2700239 / brand: atlant
product_id : 1004767 / brand: samsung
product_id : 1005135 / brand: apple
product_id : 1002544 / brand: apple
product_id : 1004659 / brand: samsung
product_id : 1306083 / brand: hp


In [ ]:
''' # df_cat을 안에 둔 경우

# surprise 모델 활용한 10개의 추천 아이템

def recomm_item_by_surprise(algo, userID, unseen_items, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 본 적이 없는 아이템에 반복 수행한 후 결과를 list로 저장
    predictions = [algo.predict(str(userID), str(item_id)) for item_id in unseen_items]

    def sortkey_est(pred):
        return pred.est

    # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]

    # top_n으로 추출된 아이템의 정보 추출. user_id, 추천 예상 평점, category_id 추출
    top_item_ids = [int(pred.iid) for pred in top_predictions]
    top_item_rating = [ pred.est for pred in top_predictions]
    top_item_brands = df_cat[df_cat.product_id.isin(top_item_ids)]['brand']

    top_item_preds = [ (id, brand, rating) for id, brand, rating in \
                        zip(top_item_ids, top_item_brands, top_item_rating)]

    return top_item_preds

unseen_items = get_unseen_surprise(df_cat, 550346297)
top_item_preds = recomm_item_by_surprise(knnbaseline_algo_i,550346297, unseen_items, top_n=10)

print('#### 10개의 추천 아이템 리스트 ####')
for top_item in top_item_preds:
    print(top_item[0], ":", top_item[1])
'''

In [52]:
## svd 

svd_algo = SVD(n_factors=50, random_state=42) # 수치 고정하여 결과가 같게

svd_algo.fit(trainset)
svd_predictions = svd_algo.test(testset)

file_name = 'svd_model'
dump.dump(file_name, algo=svd_algo)
# _, loaded_algo = dump.load(file_name)

# svd 모델 RSME, MAE
accuracy.rmse(svd_predictions)
accuracy.mae(svd_predictions)
print("Done!")

RMSE: 0.1925
MAE:  0.1342
Done!


In [53]:
# svd 모델 추천 리스트
# 유저: 548066004

unseen_items = get_unseen_surprise(df_cat, 548066004)
top_item_preds = recomm_item_by_surprise(svd_algo, df_cat, 548066004, unseen_items, top_n=10)


봤던 아이템 수: 38 추천 대상 아이템 수: 6279569 전체 아이템 수: 6721651


In [54]:
print('#### 10개의 추천 아이템 리스트 ####')
for top_item in top_item_preds:
    print("product_id :", top_item[0], '/', "brand:",top_item[1] )

#### 10개의 추천 아이템 리스트 ####
product_id : 1307067 / brand: lenovo
product_id : 1004237 / brand: apple
product_id : 1306631 / brand: hp
product_id : 1005135 / brand: apple
product_id : 2700239 / brand: atlant
product_id : 1005135 / brand: apple
product_id : 1002544 / brand: apple
product_id : 1004659 / brand: samsung
product_id : 1306083 / brand: hp
product_id : 1004659 / brand: samsung
